# Carrier Frequency Drift

This notebook demonstates the functional of the carrier frequency drift transform. The transform models the carrier having a frequency which is represented by a Gaussian random variable whose mean is the average center frequency and whose variance is parameterizable. It in effect simulates a type of frequency modulation such that frequency on average is correct but varies over time based on the randomess created from the Gaussian random variable. It simulates the error and uncertainty of the real-world parameter due to manufacturing tolerance, temperature variation and other factors.

In [ ]:
import torchsig.transforms.functional as F

import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt

A tone signal is generated to test the carrier frequency drift transform. The error is defined in parts-per-million (PPM). A larger PPM represents a larger frequency error on average. Using 0 PPM will result in no center frequency drift and a pure carrier tone.

In [ ]:
# test functional
rng = np.random.default_rng(42)

N = 10000
sample_rate = 10e6
#center_frequency = 0
center_frequency = sample_rate/N
n = np.arange(0,N)
t = n/sample_rate
tone_bb_data = np.exp(2j*np.pi*center_frequency*t)

drift_ppm=10

data_out = F.carrier_frequency_drift(
    data = tone_bb_data,
    drift_ppm = drift_ppm,
    rng = rng
)

Time-domain plots show how the frequency drift effects the complex sinusoid input. For small PPM errors the effect will may not be visible in the time domain.

In [ ]:
fig = plt.figure(figsize=(10,6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2,1,1)
ax.plot(t,np.real(tone_bb_data),label='Real, Input Tone')
ax.plot(t,np.imag(tone_bb_data),label='Imag, Input Tone')
ylim = np.max(np.abs(tone_bb_data))*1.1
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

ax = fig.add_subplot(2,1,2)
ax.plot(t,np.real(data_out),label='Real, Tone with Freq Drift')
ax.plot(t,np.imag(data_out),label='Imag, Tone with Freq Drift')
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

The frequency drift effect is more noticable in the frequency domain. The input tone is plotted against the tone with frequency drift error. The variance of the frequency parameter acts as a type of frequency modulation, thereby increasing the bandwidth of the tone. Larger PPM errors will increase the amount of frequency modulation. This effect is seen in the frequency domain by the increase in bandwidth and the raising of sidelobes beyond the unmodulated, input carrrier.

In [ ]:
win = sp.signal.windows.blackmanharris(len(tone_bb_data))
fft_size = 2**20
f = np.linspace(-0.5,0.5-(1/fft_size),fft_size)*sample_rate
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)
ax.plot(f,20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(tone_bb_data*win,fft_size)))),label='Input Tone')
ax.plot(f,20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(data_out*win,fft_size)))),'--',label='Output Tone with Freq Drift')
ax.legend(loc='upper right')
ax.set_xlim([-100000+center_frequency,100000+center_frequency])
ax.grid()
ax.set_ylabel('Magnitude (dB)')
ax.set_xlabel('Frequency (Hz)')


The instantaneous frequency and phase are calculated from the impaired signal. The time domain plot shows how the frequency varies over time on a sample-by-sample basis. The phase also varies over time although it is more difficult to see in the plot.

In [ ]:
phase = np.unwrap(np.angle(data_out))*(sample_rate/(2*np.pi))
frequency = np.diff(phase)-center_frequency

fig = plt.figure(figsize=(10,6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2,1,1)
ax.plot(t[0:-1],frequency)
ax.set_title('Instantaneous Frequency of Signal with LO Freq Drift')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.grid()

ax = fig.add_subplot(2,1,2)
ax.plot(t[0:1000],phase[0:1000])
ax.set_title('Instantaneous Phase of Signal with LO Freq Drift')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.grid()


In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.signals.builders import constellation_maps

def generate_qpsk_symbols(num_iq_samples: int = 128) -> np.ndarray:

    symbol_map = constellation_maps.all_symbol_maps['qpsk']
    symbol_index = np.random.randint(0,len(symbol_map),num_iq_samples)
    symbols = symbol_map[symbol_index]

    return symbols

An IQ plot demonstrates how the effect of the carrier frequency drift smears the constellation of a QPSK signal. A larger PPM is used in this example in order to visually demonstrate the effect. The randomness of the carrier's frequency results in a phase offset that varies on a sample-by-sample basis.

In [ ]:
# test data
N = 1024
qpsk_data = generate_qpsk_symbols(num_iq_samples = N)

# phase noise
impaired_qpsk_data = F.carrier_frequency_drift(
    data = qpsk_data,
    drift_ppm=100
)

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
ax.set_box_aspect(1)
ax.plot(np.real(impaired_qpsk_data),np.imag(impaired_qpsk_data),'x',label='With Frequency Drift')
ax.plot(np.real(qpsk_data),np.imag(qpsk_data),'o',alpha=0.5,label='Input')
ax.grid()
ax.set_title('Constellation Plot')
ax.set_xlabel('Real')
ax.set_ylabel('Imag')
ax.legend(fontsize='large', loc='center');